In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
def read_images(folder_path):
    images = []
    for img_name in os.listdir(folder_path):
        if img_name == '.DS_Store':
            continue
        img = read_img(img_name)

        if img.shape[1] > img.shape[0]:
            img = rotate_image(img)

        images.append(img)
        
    return images

def read_img(img_name):
   return cv2.imread(f'images/{img_name}')

def rotate_image(img):
    return cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)

In [3]:
original_images = read_images('images')
processed_images = original_images.copy()

In [4]:
def crop_images(images):
    for idx, img in enumerate(images):
        y, h = 0, 3050  # upper starting point, shift 
        x, w = 800, 2250  # lefter starting point, shift

        images[idx] = img[y:y+h, x:x+w]

In [5]:
crop_images(original_images)
crop_images(processed_images)

In [6]:
def extract_channels(images):
    for idx, img in enumerate(images):
        img_hsv = extract_HSV_channel(img)
        images[idx] = extract_H_channel(img_hsv)

def extract_HSV_channel(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

def extract_H_channel(img_hsv):
    return img_hsv[:, :, 0]

In [7]:
extract_channels(processed_images)

In [8]:
def kmeans_plusplus(img, num_clusters=3): 
    pixel_values = np.float32(img.flatten())
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    _, labels, centers = cv2.kmeans(pixel_values, num_clusters, None, criteria, 10, cv2.KMEANS_PP_CENTERS)
    centers = np.uint8(centers)
    labels_reshaped = labels.reshape(img.shape)
    return labels_reshaped

def second_largest_cluster_mask(labels_reshaped, num_clusters):
    masks = [cv2.inRange(labels_reshaped, cluster_id, cluster_id) for cluster_id in range(num_clusters)]
    sizes = [cv2.countNonZero(mask) for mask in masks]
    second_largest_cluster_mask = masks[sorted(zip(sizes, range(num_clusters)), reverse=True)[1][1]]

    return second_largest_cluster_mask

In [9]:
def largest_component_mask(mask):
        mask = np.uint8(mask)
        _, components = cv2.connectedComponents(mask, connectivity=8)
        return np.uint8(components == np.argmax(np.bincount(components.flat)[1:]) + 1) * 255

In [10]:
def apply_mask(img, mask):
    return cv2.bitwise_and(img, img, mask=mask)

In [11]:
def bgr_to_gray(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [12]:
def gray_to_bgr(img):
    return cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

In [13]:
def binary_threshhold(gray_img):
    _, binary_img = cv2.threshold(gray_img, 1, 255, cv2.THRESH_BINARY)
    return binary_img

In [14]:
def median_blur(stem_region, kernel_size=99):
    return cv2.medianBlur(stem_region, kernel_size)

In [15]:
def cut_stem(masked_img, stop_distance=70):
    binary_img = binary_threshhold(masked_img)  # changed this line
    start_row = None
    end_row = None

    for row in range(binary_img.shape[0]):
        current_row = binary_img[row, :]
        non_zero_indices = np.nonzero(current_row)[0]
    
        if non_zero_indices.size > 0:
            if start_row is None:
                start_row = row
            
            pixel_distance = np.max(non_zero_indices) - np.min(non_zero_indices)
            
            if pixel_distance > stop_distance:
                end_row = row
                break

    if (start_row is not None and end_row is not None and start_row < end_row):
        stem_region = masked_img[start_row:end_row, :]  # changed this line
        blurred_stem = median_blur(stem_region)

        return np.vstack((masked_img[:start_row, :], blurred_stem, masked_img[end_row:, :]))  # changed this line
    else:
        return masked_img

In [16]:
for num_clusters in range(2, 6):  
    DIR = 'processed_images'
    os.makedirs(DIR, exist_ok=True)

    for idx, img in enumerate(processed_images):
        labels_reshaped = kmeans_plusplus(img, num_clusters)
        slcm = second_largest_cluster_mask(labels_reshaped, num_clusters)
        lcm = largest_component_mask(slcm)
        gray_original = bgr_to_gray(original_images[idx])
        masked_original = apply_mask(gray_original, lcm)
        stem_cutted_gray = cut_stem(masked_original)
        stem_cutted_mask = binary_threshhold(stem_cutted_gray)
        stem_cutted = apply_mask(original_images[idx], stem_cutted_mask)

        filename = f'{DIR}/img{idx}_{num_clusters}clusters.png'
        cv2.imwrite(filename, stem_cutted)